In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("GLOP")

# Beispiel: Scheduling mit Kosten, Nutzen und Abhängigkeiten

# Gegeben:
# - activities: Liste von Aktivitäten mit Dauer, Kosten, Nutzen
# - dependencies: Liste von (Vorgänger, Nachfolger)
# - max_time: verfügbare Stunden

activities = {
    'A': {'dauer': 2, 'kosten': 3, 'nutzen': 5},
    'B': {'dauer': 3, 'kosten': 2, 'nutzen': 8},
    'C': {'dauer': 1, 'kosten': 1, 'nutzen': 2},
    'D': {'dauer': 2, 'kosten': 4, 'nutzen': 6}
}
dependencies = [('A', 'B'), ('B', 'D')]  # B nach A, D nach B
max_time = 5

# Variablen: y[act] = 1, wenn Aktivität gewählt
y = {}
for act in activities:
    y[act] = solver.BoolVar(f'y_{act}')

# Zeitbudget
solver.Add(solver.Sum([activities[act]['dauer'] * y[act] for act in activities]) <= max_time)

# Abhängigkeiten
for pre, post in dependencies:
    solver.Add(y[post] <= y[pre])

# Ziel: Gesamtnutzen maximieren
solver.Maximize(solver.Sum([activities[act]['nutzen'] * y[act] for act in activities]))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Ausgewählte Aktivitäten:")
    for act in activities:
        if y[act].solution_value() == 1:
            print(f"{act} (Dauer: {activities[act]['dauer']}, Nutzen: {activities[act]['nutzen']})")
else:
    print("Keine Lösung gefunden.")